In [7]:
import tensorflow as tf
print("hello")

hello


In [8]:
import os
import shutil

raw_dir=r'mrlEyes_2018_01'
for dirpath, dirname, filenames in os.walk(raw_dir):
   for i  in [f for f in filenames if f.endswith('.png')]:
     if i.split('_')[4]=='0':
       shutil.copy(src=dirpath+'/'+i, dst=r'closed_eyes')
     elif i.split('_')[4]=='1':
      shutil.copy(src=dirpath+'/'+i, dst=r'open_eyes')

In [5]:
closed_dir = 'closed_eyes'
open_dir = 'open_eyes'

# Count the number of images in closed and open directories
def count_images(directory):
    total_images = sum(len(files) for _, _, files in os.walk(directory))
    return total_images

closed_count = count_images(closed_dir)
open_count = count_images(open_dir)

# Print the counts
print("Number of images in closed_eyes:", closed_count)
print("Number of images in open_eyes:", open_count)

Number of images in closed_eyes: 41946
Number of images in open_eyes: 42952


In [1]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
import os
import random
import shutil

# Define paths
source_closed = 'C:\\Users\\samik\\pythonlab\\Scripts\\Drowsinessdetection\\closed_eyes'
source_open = 'C:\\Users\\samik\\pythonlab\\Scripts\\Drowsinessdetection\\open_eyes'
train_dir = 'C:\\Users\\samik\\pythonlab\\Scripts\\Drowsinessdetection\\train_data'
test_dir = 'C:\\Users\\samik\\pythonlab\\Scripts\\Drowsinessdetection\\test_data'

# Create train and test directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Function to copy files from source to destination
def copy_files(source, destination, file_list):
    for file in file_list:
        file_name = file
        dest_file_path = os.path.join(destination, file_name)
        # Check if the file already exists in the destination directory
        if os.path.exists(dest_file_path):
            # If the file exists, generate a new unique file name
            file_name, file_extension = os.path.splitext(file)
            counter = 1
            while os.path.exists(os.path.join(destination, f"{file_name}_{counter}{file_extension}")):
                counter += 1
            file_name = f"{file_name}_{counter}{file_extension}"
        shutil.copy(os.path.join(source, file), os.path.join(destination, file_name))


# List all files in the closed and open eyes directories
closed_files = os.listdir(source_closed)
open_files = os.listdir(source_open)

# Shuffle the data
random.shuffle(closed_files)
random.shuffle(open_files)

# Split the data into training and testing sets
train_closed = closed_files[:int(0.8*len(closed_files))]
train_open = open_files[:int(0.8*len(open_files))]
test_closed = closed_files[int(0.8*len(closed_files)):]
test_open = open_files[int(0.8*len(open_files)):]

# Copy files to train directory
copy_files(source_closed, os.path.join(train_dir, 'closed_eyes'), train_closed)
copy_files(source_open, os.path.join(train_dir, 'open_eyes'), train_open)

# Copy files to test directory
copy_files(source_closed, os.path.join(test_dir, 'closed_eyes'), test_closed)
copy_files(source_open, os.path.join(test_dir, 'open_eyes'), test_open)

# Now you can use the generators as before


In [4]:
# Count the number of images in train and test directories
def count_images(directory):
    total_images = sum(len(files) for _, _, files in os.walk(directory))
    return total_images

train_closed_count = count_images(os.path.join(train_dir, 'closed_eyes'))
train_open_count = count_images(os.path.join(train_dir, 'open_eyes'))
test_closed_count = count_images(os.path.join(test_dir, 'closed_eyes'))
test_open_count = count_images(os.path.join(test_dir, 'open_eyes'))

# Print the counts
print("Number of closed eye images in train:", train_closed_count)
print("Number of open eye images in train:", train_open_count)
print("Number of closed eye images in test:", test_closed_count)
print("Number of open eye images in test:", test_open_count)


Number of closed eye images in train: 33556
Number of open eye images in train: 34361
Number of closed eye images in test: 8390
Number of open eye images in test: 8591


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the ImageDataGenerator for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2  # Splitting the data into training and validation
)

# Load training data from directory
train_data = train_datagen.flow_from_directory(
    r'C:\Users\samik\pythonlab\Scripts\Drowsinessdetection\train_data',
    target_size=(80, 80),
    batch_size=8,
    class_mode='categorical',
    subset='training'  # Use the training subset
)

# Load validation data from directory
validation_data = train_datagen.flow_from_directory(
    r'C:\Users\samik\pythonlab\Scripts\Drowsinessdetection\train_data',
    target_size=(80, 80),
    batch_size=8,
    class_mode='categorical',
    subset='validation'  # Use the validation subset
)



Found 54334 images belonging to 2 classes.
Found 13583 images belonging to 2 classes.


In [7]:
# Define the ImageDataGenerator for test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Load test data from directory
test_data = test_datagen.flow_from_directory(
    r'C:\Users\samik\pythonlab\Scripts\Drowsinessdetection\test_data',
    target_size=(80, 80),
    batch_size=8,
    class_mode='categorical'
)


Found 16981 images belonging to 2 classes.


In [8]:
base_model= InceptionV3(include_top=False,weights='imagenet',input_tensor=Input(shape=(80,80,3)))
head_model=base_model.output

87910968/87910968 [==============================] - 22s 0us/step


In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
checkpoint=ModelCheckpoint(r'model checkpoint\model.h5',monitor='val_loss',save_best_only=True,verbose=3)

earlystop=EarlyStopping(monitor='val_loss',patience=7,verbose=3,restore_best_weights=True)
learning_rate=ReduceLROnPlateau(monitor='val_loss',patience=3,verbose=3)
callbacks=[checkpoint,earlystop,learning_rate]

In [10]:
!pip install -U scikit-learn 

     ---------------------------------------- 10.6/10.6 MB 3.8 MB/s eta 0:00:00
     -------------------------------------- 302.2/302.2 kB 2.7 MB/s eta 0:00:00
     ---------------------------------------- 46.2/46.2 MB 1.2 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
head_model=Flatten()(head_model)
head_model=Dense(64,activation='relu')(head_model)
head_model=Dropout(0.5)(head_model)
head_model=Dense(2,activation='softmax')(head_model)

model=Model(inputs=base_model.input,outputs=head_model)

for layer in base_model.layers:
    layer.trainable=False

In [12]:
pip install Pillow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\samik\pythonlab\Scripts\python.exe -m pip install --upgrade pip


In [17]:
import PIL
from PIL import Image

In [18]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelEncoder
batchsize=80
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,epochs=5)

C:\Users\samik\AppData\Local\Temp\ipykernel_10604\2631058467.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


Epoch 1/5
679/679 [==============================] - ETA: 0s - loss: 0.3483 - accuracy: 0.8818
Epoch 1: val_loss improved from inf to 0.34336, saving model to model checkpoint\model.h5
679/679 [==============================] - 235s 337ms/step - loss: 0.3483 - accuracy: 0.8818 - val_loss: 0.3434 - val_accuracy: 0.8676 - lr: 0.0010
Epoch 2/5
679/679 [==============================] - ETA: 0s - loss: 0.2634 - accuracy: 0.9105
Epoch 2: val_loss improved from 0.34336 to 0.28717, saving model to model checkpoint\model.h5
679/679 [==============================] - 306s 450ms/step - loss: 0.2634 - accuracy: 0.9105 - val_loss: 0.2872 - val_accuracy: 0.8913 - lr: 0.0010
Epoch 3/5
679/679 [==============================] - ETA: 0s - loss: 0.2452 - accuracy: 0.9157
Epoch 3: val_loss did not improve from 0.28717
679/679 [==============================] - 274s 404ms/step - loss: 0.2452 - accuracy: 0.9157 - val_loss: 0.2943 - val_accuracy: 0.8809 - lr: 0.0010
Epoch 4/5
679/679 [=====================

In [19]:
acc_tr,loss_tr=model.evaluate(train_data)
print(acc_tr)
print(loss_tr)

6792/6792 [==============================] - 1669s 246ms/step - loss: 0.1954 - accuracy: 0.9282
0.19544434547424316
0.9281665086746216


In [20]:
acc_vr,loss_vr=model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

C:\Users\samik\AppData\Local\Temp\ipykernel_10604\4184188003.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_vr,loss_vr=model.evaluate_generator(validation_data)


0.2764436602592468
0.8936906456947327


In [21]:
acc_te,loss_te=model.evaluate_generator(test_data)
print(acc_te)
print(loss_te)

C:\Users\samik\AppData\Local\Temp\ipykernel_10604\3010557615.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_te,loss_te=model.evaluate_generator(test_data)


0.2028217613697052
0.9274483323097229
